In [1]:
import astropy.units as u
from elk.ensemble import EnsembleLC
import numpy as np
from astroquery.simbad import Simbad
import astropy.coordinates as coord
import elk

from astropy.io import ascii
from astropy.io import fits
import os.path

from elk.ensemble import EnsembleLC
from elk.lightcurve import BasicLightcurve

from astropy.table import Table, join, MaskedColumn, vstack, Column
from matplotlib import pyplot as plt

/Users/Tobin/opt/anaconda3/envs/elk/lib/python3.10/site-packages/lightkurve/config/__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to /Users/Tobin/.lightkurve/cache. Please move all the files in the legacy directory /Users/Tobin/.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(


In [2]:
path='/Users/Tobin/Dropbox/TESS_project/HLSPs/'

gal_list=['MW', 'SMC', 'LMC']

l_of_cs=[]

for gal in gal_list:
    filenames=[os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(path+gal+"_Final_V2/")) for f in fn]

    for file in filenames:
        l_of_cs.append(elk.ensemble.from_fits(file))

In [3]:
l_of__all_lcs=[]
for i in range(len(l_of_cs)):
    for j in range(len(l_of_cs[i].lcs)):
        l_of__all_lcs.append(l_of_cs[i].lcs[j].corrected_lc)

In [6]:
len(l_of__all_lcs)

2204

In [4]:
data_augmented_lcs=[]

for i in range(len(l_of__all_lcs)):
    data_augmented_lcs.append(l_of__all_lcs[i])
    
    #Take 12 day windows in the lc:
    data_augmented_lcs.append(l_of__all_lcs[i][:int(len(l_of__all_lcs[i])/2)])
    
    data_augmented_lcs.append(l_of__all_lcs[i][int(len(l_of__all_lcs[i])/2):])

In [5]:
len(data_augmented_lcs)

6612

In [13]:
names=[]

for i in range(len(l_of_cs)):
    j=0
    while j < (len(l_of_cs[i].lcs)):
        names.append(l_of_cs[i].callable)
        names.append(l_of_cs[i].callable)
        names.append(l_of_cs[i].callable)
        
        j+=1
        
len(names)

6612

In [18]:
import numpy as np

from astroML.datasets import fetch_sdss_galaxy_images, fetch_LINEAR_sample, fetch_LINEAR_geneva
from astroML.utils import split_samples

import matplotlib.pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay, PrecisionRecallDisplay
from sklearn.gaussian_process import GaussianProcessRegressor

from tqdm import tqdm
from scipy.special import entr

In [25]:
%%time
l_of_stat_tables=[]

for i in range(len(data_augmented_lcs)):
    
    lc=BasicLightcurve(data_augmented_lcs[i]['time'], data_augmented_lcs[i]['flux'], 
                    data_augmented_lcs[i]['flux_err'], sector=99)
    
    lc.get_stats_using_defaults()

    table=lc.get_stats_table(names[i])['name',
     'rms',
     'std',
     'MAD',
     'sigmaG',
     'skewness',
     'von_neumann_ratio',
     'J_Stetson',
     'max_power',
     'freq_at_max_power',
     'n_peaks',
     'ratio_of_power_at_high_v_low_freq',
     'FAP',
     'max_autocorrelation',
     'time_of_max_autocorrelation']
    
    
    sum_LSP_power_10_7_days = np.sum(lc.periodogram[np.where((1/np.arange(0.04, 11, 0.01) < 10) &
                                                                                         (1/np.arange(0.04, 11, 0.01) > 7))])
    sum_LSP_power_7_4_days = np.sum(lc.periodogram[np.where((1/np.arange(0.04, 11, 0.01) < 7) &
                                                                                         (1/np.arange(0.04, 11, 0.01) > 4))])
    sum_LSP_power_4_1_days = np.sum(lc.periodogram[np.where((1/np.arange(0.04, 11, 0.01) < 4) &
                                                                                         (1/np.arange(0.04, 11, 0.01) > 1))])            
    sum_LSP_power_1_p5_days = np.sum(lc.periodogram[np.where((1/np.arange(0.04, 11, 0.01) < 1) &
                                                                                         (1/np.arange(0.04, 11, 0.01) > .5))])        

    entropy = entr(data_augmented_lcs[i]['flux'].value).sum()
    
    table.add_column(Column(sum_LSP_power_10_7_days), name='SumLSP_10_7_Day_Power')
    table.add_column(Column(sum_LSP_power_7_4_days), name='SumLSP_7_4_Day_Power')
    table.add_column(Column(sum_LSP_power_4_1_days), name='SumLSP_4_1_Day_Power')
    table.add_column(Column(sum_LSP_power_1_p5_days), name='SumLSP_1_p5_Day_Power')
    table.add_column(Column(entropy), name='Entropy')
    
    l_of_stat_tables.append(table)

CPU times: user 1h 33min 17s, sys: 1min 46s, total: 1h 35min 3s
Wall time: 57min 54s


In [26]:
stat_table=vstack(l_of_stat_tables)


In [27]:
stat_table

name,rms,std,MAD,sigmaG,skewness,von_neumann_ratio,J_Stetson,max_power,freq_at_max_power,n_peaks,ratio_of_power_at_high_v_low_freq,FAP,max_autocorrelation,time_of_max_autocorrelation,SumLSP_10_7_Day_Power,SumLSP_7_4_Day_Power,SumLSP_4_1_Day_Power,SumLSP_1_p5_Day_Power,Entropy
str17,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64[1],float64,float64,float64,float64,float64
NGC 5381,0.20227397545191975,0.00028663242586913503,0.00018204904335616945,0.00027032716740019386,-39762320.79645123,0.00386747638646434,11.012043362399508,0.2958842721191971,2.1800000000000006,7,35.195131675248945,4.56088215887548e-83,0.6090280071896668,0.5000039539129375,0.0882684744682719,0.5510549769665339,1.1232906308776351,0.29263235273956645,-32898062834.77195
NGC 5381,0.12860581353174044,0.0002975862445908725,0.0001867799003253956,0.0002724429021245712,-115639726.90338115,0.00721742874399377,11.199325867985667,0.2857253786955443,2.1800000000000006,2,22.51114849414844,3.3952876552198184e-38,0.3519220202673758,0.5000117633198897,0.08080550521778217,0.8876330176361887,2.1106455564409226,0.6409543335174402,-16434502050.249306
NGC 5381,0.20227397545191975,0.00027494974730113116,0.0001760566489865667,0.0002598061826987392,38791830.51857596,0.008432348981660027,10.832836728496938,0.3320229649647809,2.1800000000000006,2,63.34512451929029,1.7777047673531952e-46,0.6090280071896668,0.5000039539129375,0.24216565052982983,1.6618852997621623,2.121879951526237,0.44974226253595767,-16463560784.522644
NGC 5381,0.18321843653972633,0.000245738941990651,0.0001701871556448742,0.0002522658939011847,-32756987.980770417,0.001012114373717723,5.25277035396073,0.23854597444846268,2.1800000000000006,4,30.87554425544451,9.90665683751058e-214,0.6303671900013474,0.44444842470238655,0.28825173684391014,0.37021043620396876,1.2172146492043667,0.17228709457487934,-91041894912.14279
NGC 5381,0.16376175676775878,0.0002534884014241052,0.00016887632604312142,0.00025096998042304134,-44597560.85915224,0.0017580139075232541,5.337402035735437,0.26075298650204753,0.29000000000000004,3,19.084977912321865,9.810479911478713e-117,0.45047427687310126,0.4375102828348645,0.19316854007885273,0.7745783356310973,2.446638191161056,0.4848427133513104,-45508777674.068115
NGC 5381,0.19277891231682573,0.0002376975364376943,0.00017171613277799924,0.00025477558196146865,-21301445.912184875,0.002447399676829137,5.168786562282312,0.28095743538439083,0.29000000000000004,3,68.07649906473408,7.569536297791576e-128,0.6506501243652288,0.4444482093304032,0.6862187263521995,1.3331866974384834,2.096172191257515,0.2084774736192974,-45533117238.07469
RUPRECHT 172,0.1834981906746259,0.00013813268191284538,9.658515423227598e-05,0.0001423529350397411,-5722877.676450607,0.0017592178915821653,5.549086025349347,0.2590355085332957,2.5600000000000005,6,2.394551716018853,6.9405772961880284e-43,0.5072667443450789,0.39581298828125,0.004000690838380462,0.06179623348576854,0.7664139505131168,0.46464581399629384,-21751159551.705124
RUPRECHT 172,0.1267374136537379,0.00012791040198304837,8.954095944324614e-05,0.00013245362948005368,-574592.8288860588,0.0034659624925341363,5.1303206957816965,0.1793904799681173,5.170000000000001,5,1.8211484925674009,3.682136800344927e-12,0.37247749967226984,0.39581298828125,0.017719078474996625,0.32034569647461336,2.1286328166050366,1.0786499015421005,-10860420791.485622
RUPRECHT 172,0.1834981906746259,0.00014761891850639786,0.00010665695263045727,0.00015739651748061293,-10740246.757590875,0.003570801004023888,5.983457207801635,0.3853190685488005,2.5600000000000005,5,2.9486820506883045,2.3181594169957363e-34,0.5072667443450789,0.39581298828125,0.0007953316968404104,0.010343341987495461,1.1170595036421498,0.8554420581349976,-10890738760.219505


In [28]:
#get_ages

mw=Table.read('../Variability_Statistics/Use_MW.fits')
mw

smc=Table.read('../Variability_Statistics/Bica_Cut_down.fits')
smc

lmc=Table.read('../Variability_Statistics/Glatt_Cut_down.fits')
lmc

Name,DEC,RA,n_Name,E_B-V_,RADIUS,l_Age,Age,q_Age,Age2,Vmag,e_Vmag,SimbadName,Cross-ID,recno
,,,,mag,arcmin,,[yr],,[yr],mag,mag,,,
bytes7,float64,float64,bytes1,float64,float64,bytes1,float64,int16,float64,float64,float64,bytes17,bytes46,int32
LMC0022,-68.6725,69.59166666666665,,0.08,45.0,,8.5,1,0.0,13.11,0.14,NGC1652,"NGC1652,SL10,LW14, ESO55SC32,KMHK23",3
LMC0076,-72.39472222222223,71.69166666666666,,0.05,60.0,,8.7,1,0.0,13.51,0.12,[SL63] 37,"SL37,LW61,KMHK98",26
LMC0086,-69.37388888888889,71.93333333333332,,0.15,40.2,,8.0,1,0.0,12.26,0.12,NGC1695,"NGC1695,SL40,ESO56SC03, KMHK101",32
LMC0098,-69.11472222222221,72.26666666666665,,0.1,43.199999999999996,,8.0,1,0.0,11.41,0.12,NGC1698,"NGC1698,SL45,ESO56SC06,KMHK115",37
LMC0111,-69.75527777777778,72.47916666666666,,0.05,49.199999999999996,,7.5,1,0.0,11.41,0.09,NGC1704,"NGC1704,SL50,ESO56SC09, KMHK130",44
LMC0120,-69.985,72.65416666666665,,0.18,102.0,,7.2,1,0.0,9.469,0.11,NGC1711,"NGC1711,SL55,ESO56SC10,KMHK145",51
LMC0157,-68.91888888888889,73.225,,0.15,40.2,,8.1,2,0.0,12.58,0.12,[SL63] 75,"SL75,KMHK199",70
LMC0163,-68.21138888888889,73.2875,,0.25,37.2,,8.1,2,0.0,11.79,0.1,[SL63] 76,"SL76,KMHK206",75


In [29]:
ages=[]
for name in list(stat_table['name']):
    for i in range(len(mw)):
        if mw[i]['NAME']== name:
            ages.append(mw[i]['LOG_AGE'])
    
    for j in range(len(smc)):
        if smc[j]['SimbadName']==name:
            ages.append(smc[j]['logAge'])
            
    for k in range(len(lmc)):
        if lmc[k]['SimbadName']==name:
            ages.append(lmc[k]['Age'])
            
len(ages)

6612

In [30]:
stat_table.add_column(Column(ages), name='Lit_Clst_Age', index=1)

stat_table

name,Lit_Clst_Age,rms,std,MAD,sigmaG,skewness,von_neumann_ratio,J_Stetson,max_power,freq_at_max_power,n_peaks,ratio_of_power_at_high_v_low_freq,FAP,max_autocorrelation,time_of_max_autocorrelation,SumLSP_10_7_Day_Power,SumLSP_7_4_Day_Power,SumLSP_4_1_Day_Power,SumLSP_1_p5_Day_Power,Entropy
str17,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64[1],float64,float64,float64,float64,float64
NGC 5381,8.65,0.20227397545191975,0.00028663242586913503,0.00018204904335616945,0.00027032716740019386,-39762320.79645123,0.00386747638646434,11.012043362399508,0.2958842721191971,2.1800000000000006,7,35.195131675248945,4.56088215887548e-83,0.6090280071896668,0.5000039539129375,0.0882684744682719,0.5510549769665339,1.1232906308776351,0.29263235273956645,-32898062834.77195
NGC 5381,8.65,0.12860581353174044,0.0002975862445908725,0.0001867799003253956,0.0002724429021245712,-115639726.90338115,0.00721742874399377,11.199325867985667,0.2857253786955443,2.1800000000000006,2,22.51114849414844,3.3952876552198184e-38,0.3519220202673758,0.5000117633198897,0.08080550521778217,0.8876330176361887,2.1106455564409226,0.6409543335174402,-16434502050.249306
NGC 5381,8.65,0.20227397545191975,0.00027494974730113116,0.0001760566489865667,0.0002598061826987392,38791830.51857596,0.008432348981660027,10.832836728496938,0.3320229649647809,2.1800000000000006,2,63.34512451929029,1.7777047673531952e-46,0.6090280071896668,0.5000039539129375,0.24216565052982983,1.6618852997621623,2.121879951526237,0.44974226253595767,-16463560784.522644
NGC 5381,8.65,0.18321843653972633,0.000245738941990651,0.0001701871556448742,0.0002522658939011847,-32756987.980770417,0.001012114373717723,5.25277035396073,0.23854597444846268,2.1800000000000006,4,30.87554425544451,9.90665683751058e-214,0.6303671900013474,0.44444842470238655,0.28825173684391014,0.37021043620396876,1.2172146492043667,0.17228709457487934,-91041894912.14279
NGC 5381,8.65,0.16376175676775878,0.0002534884014241052,0.00016887632604312142,0.00025096998042304134,-44597560.85915224,0.0017580139075232541,5.337402035735437,0.26075298650204753,0.29000000000000004,3,19.084977912321865,9.810479911478713e-117,0.45047427687310126,0.4375102828348645,0.19316854007885273,0.7745783356310973,2.446638191161056,0.4848427133513104,-45508777674.068115
NGC 5381,8.65,0.19277891231682573,0.0002376975364376943,0.00017171613277799924,0.00025477558196146865,-21301445.912184875,0.002447399676829137,5.168786562282312,0.28095743538439083,0.29000000000000004,3,68.07649906473408,7.569536297791576e-128,0.6506501243652288,0.4444482093304032,0.6862187263521995,1.3331866974384834,2.096172191257515,0.2084774736192974,-45533117238.07469
RUPRECHT 172,9.065,0.1834981906746259,0.00013813268191284538,9.658515423227598e-05,0.0001423529350397411,-5722877.676450607,0.0017592178915821653,5.549086025349347,0.2590355085332957,2.5600000000000005,6,2.394551716018853,6.9405772961880284e-43,0.5072667443450789,0.39581298828125,0.004000690838380462,0.06179623348576854,0.7664139505131168,0.46464581399629384,-21751159551.705124
RUPRECHT 172,9.065,0.1267374136537379,0.00012791040198304837,8.954095944324614e-05,0.00013245362948005368,-574592.8288860588,0.0034659624925341363,5.1303206957816965,0.1793904799681173,5.170000000000001,5,1.8211484925674009,3.682136800344927e-12,0.37247749967226984,0.39581298828125,0.017719078474996625,0.32034569647461336,2.1286328166050366,1.0786499015421005,-10860420791.485622
RUPRECHT 172,9.065,0.1834981906746259,0.00014761891850639786,0.00010665695263045727,0.00015739651748061293,-10740246.757590875,0.003570801004023888,5.983457207801635,0.3853190685488005,2.5600000000000005,5,2.9486820506883045,2.3181594169957363e-34,0.5072667443450789,0.39581298828125,0.0007953316968404104,0.010343341987495461,1.1170595036421498,0.8554420581349976,-10890738760.219505


In [31]:
stat_table.write('Stats_Table_For_Data_augmented_lcs.fits', overwrite=True)